In [1]:
# Install library yang dibutuhkan (biasanya pandas & requests sudah ada, tapi untuk jaga-jaga)
!pip install pandas requests

In [2]:
import requests
import pandas as pd
from datetime import datetime
import os

# Konfigurasi
COINS = ['bitcoin', 'ethereum', 'solana']
CURRENCY = 'usd'
FILENAME = 'crypto_history.csv' # File ini akan muncul di folder file Anda

print("Library loaded & Config set.")

Library loaded & Config set.


In [3]:
def extract_data(coin_ids):
    """Mengambil harga live dari CoinGecko API"""
    ids_string = ",".join(coin_ids)
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={ids_string}&vs_currencies={CURRENCY}"
    
    try:
        response = requests.get(url)
        response.raise_for_status() # Cek jika ada error koneksi
        data = response.json()
        print("✅ Extract berhasil")
        return data
    except Exception as e:
        print(f"❌ Extract gagal: {e}")
        return None

In [4]:
def transform_data(raw_data):
    """Mengubah JSON menjadi DataFrame dan menambah kolom Waktu"""
    if not raw_data:
        return None
    
    rows = []
    waktu_skrg = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    for coin, details in raw_data.items():
        row = {
            'timestamp': waktu_skrg,
            'coin': coin,
            'price_usd': details['usd']
        }
        rows.append(row)
        
    df = pd.DataFrame(rows)
    print(f"✅ Transform berhasil: {len(df)} data diproses")
    return df

In [5]:
def load_data(df_new, filename):
    """Menyimpan data ke CSV (Append Mode)"""
    if df_new is None:
        print("Data kosong, skip loading.")
        return

    # Cek apakah file sudah ada sebelumnya
    if os.path.exists(filename):
        # Jika ada, baca dulu (opsional) lalu append tanpa header
        df_new.to_csv(filename, mode='a', header=False, index=False)
        print(f"✅ Data ditambahkan ke {filename}")
    else:
        # Jika belum ada, buat file baru dengan header
        df_new.to_csv(filename, mode='w', header=True, index=False)
        print(f"✅ File baru dibuat: {filename}")

In [6]:
# Menjalankan ETL
print("--- Mulai Proses ETL ---")

# 1. Extract
raw_data = extract_data(COINS)

# 2. Transform
df_clean = transform_data(raw_data)

# 3. Load
load_data(df_clean, FILENAME)

# (Opsional) Tampilkan 5 data terakhir untuk cek di Laporan
if os.path.exists(FILENAME):
    print("\nPreview Data Terkini:")
    df_all = pd.read_csv(FILENAME)
    print(df_all.tail())

print("--- ETL Selesai ---")

--- Mulai Proses ETL ---
✅ Extract berhasil
✅ Transform berhasil: 3 data diproses
✅ Data ditambahkan ke crypto_history.csv

Preview Data Terkini:
             timestamp      coin  price_usd
1  2025-12-06 10:35:24  ethereum    3031.07
2  2025-12-06 10:35:24    solana     132.58
3  2025-12-08 00:07:18   bitcoin   90367.00
4  2025-12-08 00:07:18  ethereum    3058.70
5  2025-12-08 00:07:18    solana     132.01
--- ETL Selesai ---
